In [9]:
#%autoreload
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.layers import Merge 
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint
#import imp
#imp.reload(parse_data)

In [2]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
   # return np.linalg.norm(y_pred-y_true)/np.linalg.norm(y_true)

In [17]:
#rgb
model_left = Sequential()
rgb_model_vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(299,299,3))
model_left.add(rgb_model_vgg16)
model_left.get_layer(name='vgg16_input').name='vgg16_left_input'
model_left.get_layer(name='vgg16').name='vgg16_left'
#depth
model_right = Sequential()
depth_model_vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(299,299,3))
model_right.add(depth_model_vgg16)
#merge
model = Sequential()
model.add(Merge([model_left,model_right], mode='concat'))  
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation = 'relu'))          

model.compile(loss='mean_squared_error', optimizer=SGD(lr=1e-5, momentum=0.9),
            metrics=[root_mean_squared_error])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_5 (Merge)              (None, 9, 9, 1024)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 82944)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              84935680  
_________________________________________________________________
dense_11 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 1539      
Total params: 114,891,395
Trainable params: 114,891,395
Non-trainable params: 0
_________________________________________________________________


/Library/Python/2.7/site-packages/ipykernel_launcher.py:13: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]


In [18]:
object_path = "train/48/"
dev_path = "dev/48/"
filename ="vgg"
rescale=1. / 255
img_width, img_height = 299, 299

In [22]:
dev_num = 7
x_dev = np.empty([0,299,299,3])
d_dev = np.empty([0,299,299,3])
y_dev = np.empty([0,3])
for k in range(dev_num):
    x_dev_tmp = np.load(dev_path+str(60+k)+"_x.npy")
    d_dev_tmp = np.load(dev_path+str(60+k)+"_d.npy")
    x_dev = np.vstack([x_dev, x_dev_tmp])
    d_dev = np.vstack([d_dev, d_dev_tmp])
    tmp2 = np.load(dev_path+str(60+k)+"_y.npy")
    y_dev = np.vstack([y_dev, tmp2])
print y_dev.shape
print X_dev.shape

IOError: [Errno 2] No such file or directory: 'dev/48/60_x.npy'

In [23]:
dev_loss = model.evaluate(x = [x_dev, d_dev], y = y_dev)
print "----val_loss & l2_error----:"
print dev_loss

----val_loss & l2_error----:
[]


In [24]:
def train(model):
    filename = "vgg2d_"
    fh = open(filename + 'report.txt','w')
    csv_logger = CSVLogger('logvgg.csv', append=True, separator=';') 
    f_train = open(filename + 'train_report.txt',"a", 0)
    f_train_step = open(filename + 'step_report.txt',"a", 0)
    f_dev = open(filename + 'dev_report.txt',"a", 0)
    batch = 32
    epochs = 20
    batch_num = 60
    rescale=1. / 255
    filepath_best="/home/zhouzixuan/proj/data/weights.best.hdf5"
    if os.path.exists(filepath_best):
        model.load_weights(filepath_best)
        print "load weight success!"
    for e in range(4,epochs):
        print('Epoch', e)
        for b in range(batch_num):
            x_batch = np.load(object_path+str(b)+"_x.npy")
            d_batch = np.load(object_path+str(b)+"_d.npy")     
            y_batch = np.load(object_path+str(b)+"_y.npy")            
            checkpoint = ModelCheckpoint(filepath=filepath_best,monitor='loss', verbose=1,save_best_only=True, mode='min')
            res = model.fit([x_batch, d_batch], y_batch,callbacks=[checkpoint],verbose=0)
            print str(res.history)
            f_train_step.write(str(res.history))
            f_train_step.write("\n")
        train_loss = model.evaluate([x_batch, d_batch], y = y_batch)
        f_train.write(str(train_loss))
        dev_loss = model.evaluate(x = [x_dev, y_dev], y = y_dev)
        f_dev.write(str(dev_loss))
        print "----val_loss & l2_error----:"
        print dev_loss
    f_train.close()
    f_train_step.close()
    f_dev.close()

In [ ]:
train(model)